#### ⏰ Até 15/02 ⏰ 1ª FASE - EXTRAÇÃO DOS DADOS DA ESCALA DE SERVIÇO 

In [ ]:
from docx import Document
import pandas as pd

# função que abre e extrai todas as tabelas de um docx
def extrair_tabelas(caminhoDocx):
    todasTabelas = []

    doc = Document(caminhoDocx)
    
    for tabela_idx, tabela in enumerate(doc.tables):
        tabelaAtual = []
        
        for row_idx, row in enumerate(tabela.rows):
            cells = [cell.text.strip() for cell in row.cells]
            tabelaAtual.append(cells)

        todasTabelas.append(tabelaAtual)

    return todasTabelas

# coloca o nome do docx
tabelas = extrair_tabelas('08.02.2024 ESCALA DE SERVIÇO - QUINTA-FEIRA.docx')

# criação dos DataFrames a partir da extração em docx
for idx, tabela in enumerate(tabelas):
    dfEquipes = pd.DataFrame(tabela)
    display(dfEquipes)    
  



# DESCONSIDERE A PARTIR DESTA LINHA - SÃO FRAGMENTOS DE CÓDIGOS USADOS APENAS PARA FAZER TESTE
    
# from tabula import read_pdf, convert_into
# import pandas as pd

# # # #conversão de arquivos PDF
# # convert_into('CPU_1QTU.pdf', 'livro_1QTU', pages='all', output_format='csv', stream=True)
# # convert_into('CPU_2QTU.pdf', 'livro_2QTU', pages='all', output_format='csv', stream=True)
# # convert_into('EscalaServico.pdf', 'escala', pages='all', output_format='csv', stream=True)


# tabelaEscalaAtual = read_pdf("EscalaServico.pdf", pages="all")
# # print(len(lista_tabelas)) - apenas informa a quantidade de tabelas 
# for tabela in tabelaEscalaAtual:
#     display(tabela)

# # Extração da Escala do Livro do CPU - 1º QTU e 2º QTU
# tabelasEscalaNova = read_pdf('EscalaServico_NovoModelo.pdf', pages='all')
# for tabela in tabelasEscalaNova:
#     display(tabela)




#### ⏰ Até 20/02 ⏰ 2ª FASE - EXTRAÇÃO DOS DADOS DO CHECKLIST 

#### ⏰ Até 25/02 ⏰ 3ª FASE - COMPARAÇÃO DADOS DA ESCALA E CHECKLIST EXECUTADOS 

#### ⏰ Até 1ª/03 ⏰ 4ª FASE - RELATÓRIO AO CHEFE DA 4ª SEÇÃO 